In [6]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
import json
import random
import numpy as np
import pickle

lemmatizer = WordNetLemmatizer()

words = []
classes = []
word_tag = []

ignoring_chars = ['?', '!']

data = open('intents.json', encoding = "utf8").read()
intents = json.loads(data)

for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenizing
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        word_tag.append((w, intent['tag']))

        #adding to classes
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

#for w in words:
#  if w not in ignoring_chars:
#    words_lemmatized = lemmatizer.lemmatize(w.lower())
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignoring_chars]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

#serilaization
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

train_data = []
output_data = [0 for element in range(len(classes))]

#create our training data
for w_tag in word_tag:
  bag_of_words = []
  pattern_words = w_tag[0]
  #for pattern_word in pattern_words:
  #  pattern_words_lemmatized = lemmatizer.lemmatize(pattern_word.lower())
  pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
  #if word found in current pattern, create bag of words array of 1, otherwise 0
  for word in words:
    if word in pattern_words:
        bag_of_words.append(1)
    else:
      bag_of_words.append(0)
  
  #output is 1 for current tag
  output_row = list(output_data)
  output_row[classes.index(w_tag[1])] = 1

  train_data.append([bag_of_words, output_row])

random.shuffle(train_data)
train_data = np.array(train_data)

X_train = list(train_data[:,0]) #patterns
Y_train = list(train_data[:,1]) #responses

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
<ipython-input-6-2db73b0599e8>:70: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  train_data = np.array(train_data)


In [7]:
#creating model
model = Sequential()
model.add(Dense(128, input_shape = (len(X_train[0]),), activation = 'relu')) #first layer with 128 neurons
model.add(Dropout(0.3))
model.add(Dense(64, activation = 'relu')) #second layer with 128/2 neurons
model.add(Dropout(0.3))
model.add(Dense(len(Y_train[0]), activation = 'softmax')) #third layer with Y_train[0] neurons

# Compile model
model.compile(optimizer="Adam", loss="mse", metrics=["mae"])

#fitting and saving the model
fitted_model = model.fit(np.array(X_train), np.array(Y_train), epochs = 350, batch_size = 5, verbose = 1)
model.save('model.h5', fitted_model)

print("model is created.")

Epoch 1/350
157/157 [==============================] - 1s 3ms/step - loss: 0.0062 - mae: 0.0123
Epoch 2/350
157/157 [==============================] - 1s 3ms/step - loss: 0.0061 - mae: 0.0123
Epoch 3/350
157/157 [==============================] - 0s 3ms/step - loss: 0.0059 - mae: 0.0120
Epoch 4/350
157/157 [==============================] - 1s 3ms/step - loss: 0.0054 - mae: 0.0111
Epoch 5/350
157/157 [==============================] - 0s 3ms/step - loss: 0.0050 - mae: 0.0103
Epoch 6/350
157/157 [==============================] - 1s 3ms/step - loss: 0.0046 - mae: 0.0097
Epoch 7/350
157/157 [==============================] - 1s 5ms/step - loss: 0.0043 - mae: 0.0091
Epoch 8/350
157/157 [==============================] - 1s 5ms/step - loss: 0.0040 - mae: 0.0085
Epoch 9/350
157/157 [==============================] - 1s 5ms/step - loss: 0.0037 - mae: 0.0079
Epoch 10/350
157/157 [==============================] - 1s 5ms/step - loss: 0.0034 - mae: 0.0073
Epoch 11/350
157/157 [=================

In [21]:
import nltk
import random
import json
import pickle
import numpy as np
import time

from nltk.stem import WordNetLemmatizer
from keras.models import load_model

lemmatizer = WordNetLemmatizer()
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

model = load_model('model.h5')
intents = json.loads(open('intents.json',encoding="utf8").read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))
totalResponseTime = 0
numberOfResponses = 0

def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(msg):
    KeyWords = ["covid-19", "covid19", "coronavirus", "corona virus", "covid", "corona", "mask", "masks",
                "bivalent", "antiviral treatments", "antiviral treatment", "paxlovid", "total cases in canada", 
                "statistics report", "statistics", "total cases", "sanitizer", "soap", "unauthorized vaccine",
                "hi", "hey", "hello", "greetings", "help", "what can you do", "what you can do",
                "bye", "see you later", "goodbye", "have a good day", "have a nice day", "have a good one",
                "see you soon", "have a good night", "thanks", "thank you", "awsome", "booster", "omicron",
                "flu", "flushot", "influenza", "monkeypox", "monkey pox", "children vaccine", "vaccine for children", 
                "vaccine for my 5-11 year old", "vaccine for my 6 months to 5-year-old", 
                "pay to receive the vaccine", "pay for vaccine", "pay for the vaccine", "vaccine cost",
                "pfizer", "moderna", "pregnant", "where is north york general hospital located", 
                "where is nygh located", "location of north york general hospital", 
                "location of nygh", "what is nygh location", "what is north york general hospital location",
                "helpline numbers", "on which  number i should call", "hospital number", "who can i call",
                "call", "nygh hospital number", "hospital contact", "how do i contact the hospital", 
                "how do i talk to a doctor","how do i talk to a professional", "contact hospital", "contact pharmacy"
                "what are the travel guidelines", "what are the different approved vaccines",
                "prophylaxis", "what if my pet becomes sick while i am isolating", "vaccination", "walk-in clinics",
                "walk in clinics", "vaccine", "pet becomes sick while i am isolating", "north york cough, cold and covid test clinic",
                "can i bring a support person or family member to my appointment"]
    helper = set()
    lowered_case = msg.lower()
    for KeyWord in KeyWords:
      helper.add(lowered_case.find(KeyWord))
    
    # for non-related questions
    if len(helper) == 1:
      return "Sorry, I don't understand your question. I can only answer questions related to Monkeypox, Influenza (Flu), and Covid-19. If you have any other questions, please contact NYGH's Pharmacy at (416)756-6666 or email them through NYGHPharmacy@nygh.on.ca"
 
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

def startTimer():
    start = time.time()
    return start

def endTimer():
    end = time.time()
    return end

def calculateAverageTime(start, end):
    global totalResponseTime
    global numberOfResponses

    difference = end - start
    numberOfResponses = numberOfResponses + 1
    totalResponseTime = round(totalResponseTime + difference, 5)
    averageResponseTime = totalResponseTime / numberOfResponses

    print("New Average Response Time: ", averageResponseTime * 1000, "ms")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [20]:
while True:
  user_response = input()
  print(chatbot_response(user_response))
  print("\nDid I correctly answer your question?")
  yes_no_response = input()
  str1 = "yes"
  index = yes_no_response.lower().find(str1.lower())
  if index != -1:
    print("\nGreat! Is there anything else I can help you with? You can answer with yes or no...")
    yes_no_response_next = input()
    str2 = "yes"
    index2 = yes_no_response_next.lower().find(str2.lower())
    if index2 != -1:
      print("\nPlease type in your next question...")
      continue;
    else:
      print("\nAlright. See you later.")
      break;
  else:
    print("\nAlright. You can either ask your question again from me or contact NYGH's Pharmacy at (416)756-6666 or email through NYGHPharmacy@nygh.on.ca\n")

what is monkeypox
1/1 [==============================] - 0s 40ms/step
Monkeypox is usually a self-limited viral infection with a rash that may be painful. Most people recover on their own after a few weeks. In some circumstances, people can become very sick and could die.

Did I correctly answer your question?
yes

Great! Is there anything else I can help you with? You can answer with yes or no...
yes

Please type in your question...
what is flu
Sorry, I don't understand your question. I can only answer questions related to Monkeypox, Influenza (Flu), and Covid-19. If you have any other questions, please contact NYGH's Pharmacy at (416)756-6666 or email them through NYGHPharmacy@nygh.on.ca

Did I correctly answer your question?
no

Alright. You can either ask your question again from me or contact NYGH's Pharmacy at (416)756-6666 or email through NYGHPharmacy@nygh.on.ca

what is covid-19
1/1 [==============================] - 0s 38ms/step
Coronaviruses or COVID-19 is a large family of 